In [86]:
import numpy as np
import rasterio
import os

npver = np.__version__
rasteriover = rasterio.__version__

print(f"Numpy Version = {npver}") #Numpy Version = 2.3.5
print(f"Rasterio Version = {rasteriover}") #Rasterio Version = 1.4.3

#fuzzy_AR()
fuzzy_Elev()


Numpy Version = 2.3.5
Rasterio Version = 1.4.3
Fuzzy rasters for elev starting
raster_min = 45.785526275634766
raster_max = 1399.4171142578125
Fuzzy raster saved: Fuzzy_Layers\elev_fuzzy_low.tif
Fuzzy raster saved: Fuzzy_Layers\elev_fuzzy_moderate.tif
Fuzzy raster saved: Fuzzy_Layers\elev_fuzzy_high.tif
Fuzzy rasters for elev completed


low: 1.0 from 0% up to 20%, linear decline up to 40%  
moderate: linear increase from 30% to 40%, 1.0 from 40% to 60%, decline from 60% to 70%  
high:  increase from 0 at 60% to 80%, plateau at 80% to 100%

Compare rise and fall using minimum instead of switchcase/nested if/else.
NumPy operations like (x - a)/(b - a), np.minimum, and np.clip are implemented in compiled C under the hood.
They operate on the entire array at once without Python loops or conditionals.

In [87]:
# --------- Helper function for trapezoidal membership ---------
def trapezoid(x, a, b, c, d):
    """
    Vectorized trapezoidal membership function.
    x: scalar or numpy array
    a, b: start and top-left of trapezoid
    c, d: top-right and end of trapezoid
    Returns same shape as x.
    """
    x = np.asarray(x, dtype=float)

    # Rising edge
    rise = np.zeros_like(x)
    if b != a:
        rise = (x - a) / (b - a)
    else:
        rise[x >= b] = 1.0  # vertical rise, there is no rise

    # Falling edge
    fall = np.zeros_like(x)
    if d != c:
        fall = (d - x) / (d - c)
    else: 
        fall[x <= c] = 1.0  # vertical fall, there is no fall

    # Compute membership: min of rising, plateau=1, falling. 
    # Elements outisde rise will be 1.0, which means it will choose fall value if less than 1, or keep 1 if plateau.
    membership = np.minimum(np.minimum(rise, 1.0), fall)

    # Clip to [0,1]
    return np.clip(membership, 0.0, 1.0)


In [88]:
# --------- Annual Rainfall Layer ------------
# --------- 2-2-1_PH_AR-ANN-RF-1986-2005_MO_2023
# --------- Read raster ---------
def fuzzy_AR():
    raster_path = rf"Component_Layers\AR-86-05-Marikina_WS.tif"
    layer = "AR"
    print(f"Fuzzy rasters for {layer} starting")

    with rasterio.open(raster_path) as src:
        raster = src.read(1).astype(float)  # Read first band
        profile = src.profile

    # --------- Normalize raster to 0-1 ---------
    nodata = -9999
    raster = np.where(raster == nodata, np.nan, raster)

    raster_min = np.nanmin(raster)
    raster_max = np.nanmax(raster)
    print(f"raster_min = {raster_min}")
    print(f"raster_max = {raster_max}")

    # --------- Normalize raster ---------
    denom = raster_max - raster_min
    if denom == 0:
        print("WARNING: raster_min == raster_max — normalization forced to 0")
        raster_norm = np.zeros_like(raster, dtype=float)
    else:
        raster_norm = (raster - raster_min) / denom
    
    # --------- Compute fuzzy memberships ---------
    low = trapezoid(raster_norm, 0.0, 0.0, 0.2, 0.4)
    moderate = trapezoid(raster_norm, 0.2, 0.4, 0.6, 0.8)
    high = trapezoid(raster_norm, 0.6, 0.8, 1.0, 1.0)


    # --------- Save fuzzy rasters ---------
    for layer, name, data in zip([layer]*3, ["low", "moderate", "high"], [low, moderate, high]):
        out_path = rf"Fuzzy_Layers\{layer}_fuzzy_{name}.tif"
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(data.astype(rasterio.float32), 1)
        print(f"Fuzzy raster saved: {out_path}")

    print(f"Fuzzy rasters for {layer} completed")

In [ ]:
# --------- Elevation Layer ------------
# --------- COG_SRTM90m_CGIAR-CSI_2018 = elevation-upper-marikina-watershed-SRTM.
# --------- DEM_ifsar_30m_phil.tif = NAMRIA 2013
# --------- Read raster ---------
def fuzzy_Elev():
    raster_path = rf"Component_Layers\elevation-upper-marikina-watershed-DEM.tif"
    layer = "elev"
    print(f"Fuzzy rasters for {layer} starting")
    
    with rasterio.open(raster_path) as src:
        raster = src.read(1).astype(float)  # Read first band
        profile = src.profile

    # --------- Normalize raster to 0-1 ---------
    nodata = -9999
    raster = np.where(raster == nodata, np.nan, raster)

    raster_min = np.nanmin(raster)
    raster_max = np.nanmax(raster)
    print(f"raster_min = {raster_min}")
    print(f"raster_max = {raster_max}")

    # --------- Normalize raster ---------
    denom = raster_max - raster_min
    if denom == 0:
        print("WARNING: raster_min == raster_max — normalization forced to 0")
        raster_norm = np.zeros_like(raster, dtype=float)
    else:
        raster_norm = (raster - raster_min) / denom

    # Replace nan with a placeholder (optional)
    #raster_to_save = np.where(np.isnan(raster_norm), -9999, raster_norm)

    # Save as text
    #np.savetxt("normalized_raster.txt", raster_to_save, fmt="%.4f")
    
    # --------- Compute fuzzy memberships ---------
    low = trapezoid(raster_norm, 0.0, 0.0, 0.2, 0.4)
    moderate = trapezoid(raster_norm, 0.2, 0.4, 0.6, 0.8)
    high = trapezoid(raster_norm, 0.6, 0.8, 1.0, 1.0)


    # --------- Save fuzzy rasters ---------
    for layer, name, data in zip([layer]*3, ["low", "moderate", "high"], [low, moderate, high]):
        out_path = rf"Fuzzy_Layers\{layer}_fuzzy_{name}.tif"
        profile.update(dtype=rasterio.float32, count=1)
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(data.astype(rasterio.float32), 1)
        print(f"Fuzzy raster saved: {out_path}")

    print(f"Fuzzy rasters for {layer} completed")


Fuzzy Gamma Operator


In [ ]:
import numpy as np

# Example: two fuzzy membership rasters
# elev_high = ... (numpy array 0-1)
# rain_high = ... (numpy array 0-1)
# USE FUZZY GAMMA OPERATOR

gamma = 0.5  # compromise between AND (Very strict) (0) and OR (Very Relaxed) (1) 

# Fuzzy AND (algebraic product)
fuzzy_and = elev_high * rain_high

# Fuzzy OR (algebraic sum)
fuzzy_or = elev_high + rain_high - (elev_high * rain_high)

# Fuzzy Gamma combination
suitability = (fuzzy_and ** (1 - gamma)) * (fuzzy_or ** gamma)

# Optional: clip to 0-1 for safety
suitability = np.clip(suitability, 0, 1)

def trapezoid2(x, a, b, c, d):
    return np.clip(
        np.minimum(
            np.minimum((x - a) / (b - a + 1e-9), 1),
            (d - x) / (d - c + 1e-9)
        ),
        0, 1
    )

suit_low  = trapezoid(suitability, 0.0, 0.0, 0.2, 0.4)
suit_mod  = trapezoid(suitability, 0.2, 0.4, 0.6, 0.8)
suit_high = trapezoid(suitability, 0.6, 0.8, 1.0, 1.0)

stack = np.stack([suit_low, suit_mod, suit_high], axis=-1)

# class labels: 0=low, 1=moderate, 2=high
class_raster = np.argmax(stack, axis=-1)